In [1]:
import gradio as gr
from transformers import pipeline
import numpy as np
import openai
import warnings
import os
from dotenv import load_dotenv
import pandas as pd
load_dotenv()

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

warnings.filterwarnings('ignore')

C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
# Preference
# Initialize the ASR pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe_and_save(audio):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))
    transcription = transcriber({"sampling_rate": sr, "raw": y})["text"]

    # Write the transcription to a file
    with open("transcription_movie_selection.txt", "w") as file:
        file.write(transcription)
    
    return transcription


# Create the Gradio interface
demo = gr.Interface(
    transcribe_and_save,
    gr.Audio(sources=["microphone"], label="Tell me how your day was and what things you enjoy in movies!"),
    "text",
)

demo.launch(share=False)
#change to True for outside URL


Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


In [3]:
#pure movie selection analysis with OpenAI
#preprocess txt file

with open('transcription_movie_selection.txt', 'r', encoding="utf-8") as file:
    text = file.read()


In [5]:
# Function time

def get_preference(question):
    messages = [
        {"role": "system", "content": "Determine the emotions of the text as anger, disgust, fear, joy, neutral, sadness, or suprise and based on the provided text select three of the following genres: Action, Adventure, Animation, Comedy, Crime, Documentary, Drama, Family, Fantasy, History, Horror, Music, Mystery, Romance, Science Fiction, TV Movie, Thriller, War, and Western."},
        {"role": "user", "content": question}
    ]
    
    # Use the chat completions method
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=messages
    )
    
    # Extract the answer from the response
    answer = response.choices[0].message.content
    return answer

def analyze_document(filename):
    with open(filename, 'r') as file:
        text = file.read()
    preference = get_preference(text)
    print(f"{preference}")

def analyze_document(filename, movie_output):
    with open(filename, 'r') as file:
        text = file.read()
    preference = get_preference(text)
    
    # Write the output to a text file
    with open(movie_output, 'w') as outfile:
        outfile.write(f"{preference}\n")

analyze_document('transcription_movie_selection.txt', 'movie_output.txt')

with open('movie_output.txt') as file:
    lines = file.readlines()

# Display the lines
print(text)
for line in lines:
    print(line.strip())


 So this works and let's go there and then go there and there.
Emotions: Neutral
Genres: Adventure, Mystery, Action


In [6]:
#Analysis tool

# Read the recommendation_ready.csv file into the dataframe for use
movie_predictions_df = pd.read_csv("recommendation_ready.csv")

In [7]:
movie_predictions_df.head()

,Unnamed: 0,genre_ids,movie_id,popularity,release_date,title,clean_reviews,type_Action,type_Adventure,type_Animation,...,label,score,anger,disgust,fear,joy,neutral,sadness,surprise,poster_path
0,0,"[878, 12]",693134,4534.956,2024-02-27,Dune: Part Two,dune part two surpasses even highest expectati...,0,1,0,...,joy,0.934813,0.002814,0.001023,0.005133,0.934813,0.037235,0.002196,0.016785,/1pdfLvkbY9ohJlCjQH2CZjjYVvJ.jpg
1,1,"[16, 28, 12, 35, 10751]",1011985,3214.314,2024-03-02,Kung Fu Panda 4,kung fu panda isnt best kung fu panda film eve...,1,1,1,...,sadness,0.598701,0.002053,0.006893,0.003863,0.068874,0.242003,0.598701,0.077614,/f7QBvIzoWSJw3jWPGnZBc5vwQ0l.jpg
2,2,"[28, 878, 12, 14]",823464,2297.340,2024-03-27,Godzilla x Kong: The New Empire,godzilla x kong new empire version previous in...,1,1,0,...,neutral,0.559732,0.173443,0.093400,0.059591,0.041290,0.559732,0.035302,0.037241,/tMefBSflR6PGQLv7WvFPpKLZkyk.jpg
3,3,"[28, 53]",359410,1134.726,2024-03-08,Road House,poor scenario story add ending also convincing...,1,0,0,...,joy,0.893423,0.002555,0.003341,0.002124,0.893423,0.077729,0.005483,0.015345,/bXi6IQiQDHD00JFio5ZSZOeRSBh.jpg
4,4,"[28, 14]",634492,921.140,2024-02-14,Madame Web,start heavily pregnant woman deep peruvian jun...,1,0,0,...,neutral,0.433036,0.070495,0.246801,0.108896,0.004469,0.433036,0.016794,0.119508,/rULWuutDcN5NvtiZi4FRPzRYWSh.jpg


In [8]:
movie_predictions_df.head()

,Unnamed: 0,genre_ids,movie_id,popularity,release_date,title,clean_reviews,type_Action,type_Adventure,type_Animation,...,label,score,anger,disgust,fear,joy,neutral,sadness,surprise,poster_path
0,0,"[878, 12]",693134,4534.956,2024-02-27,Dune: Part Two,dune part two surpasses even highest expectati...,0,1,0,...,joy,0.934813,0.002814,0.001023,0.005133,0.934813,0.037235,0.002196,0.016785,/1pdfLvkbY9ohJlCjQH2CZjjYVvJ.jpg
1,1,"[16, 28, 12, 35, 10751]",1011985,3214.314,2024-03-02,Kung Fu Panda 4,kung fu panda isnt best kung fu panda film eve...,1,1,1,...,sadness,0.598701,0.002053,0.006893,0.003863,0.068874,0.242003,0.598701,0.077614,/f7QBvIzoWSJw3jWPGnZBc5vwQ0l.jpg
2,2,"[28, 878, 12, 14]",823464,2297.340,2024-03-27,Godzilla x Kong: The New Empire,godzilla x kong new empire version previous in...,1,1,0,...,neutral,0.559732,0.173443,0.093400,0.059591,0.041290,0.559732,0.035302,0.037241,/tMefBSflR6PGQLv7WvFPpKLZkyk.jpg
3,3,"[28, 53]",359410,1134.726,2024-03-08,Road House,poor scenario story add ending also convincing...,1,0,0,...,joy,0.893423,0.002555,0.003341,0.002124,0.893423,0.077729,0.005483,0.015345,/bXi6IQiQDHD00JFio5ZSZOeRSBh.jpg
4,4,"[28, 14]",634492,921.140,2024-02-14,Madame Web,start heavily pregnant woman deep peruvian jun...,1,0,0,...,neutral,0.433036,0.070495,0.246801,0.108896,0.004469,0.433036,0.016794,0.119508,/rULWuutDcN5NvtiZi4FRPzRYWSh.jpg


In [9]:
def recommend_movies(file):
    # Read the file and parse the emotion and genres
    with open(file.name, 'r') as f:
        lines = f.readlines()

    # Extract emotion and genres from the file content
    emotion = lines[0].split(': ')[1].strip().lower() # Convert emotion to lowercase
    genres = [genre.strip().lower() for genre in lines[1].split(': ')[1].strip().split(', ')] # Convert each genre to lowercase

    # Filter movies based on the predicted emotion
    emotion_movies = movie_predictions_df[movie_predictions_df['label'].str.lower() == emotion]

    base_url = "https://image.tmdb.org/t/p/"
    file_size = "w300"


    # If no movies match the emotion, return a message
    if emotion_movies.empty:
        return [{"message": "No movies match your emotion preference."}]

    # Further filter movies based on the selected genres
    genre_movies = emotion_movies.copy()
    for genre in genres:
        # Adjust the genre name to match the DataFrame column format
        genre_column = f"type_{genre.capitalize().replace(' ', '_')}"
        genre_movies = genre_movies[genre_movies[genre_column] == 1]

    # If no movies match the genres, return a message
    if genre_movies.empty:
        return [{"message": "No movies match your genre preferences."}]

    # Rank movies based on the number of criteria they match and emotion score
    ranked_movies = []
    for index, row in genre_movies.iterrows():
        # Add the emotion score to the match count
        match_count = row[emotion]

        # Construct the poster URL
        poster_url = f"{base_url}{file_size}{row['poster_path']}"
        # Add the movie to the ranked list
        ranked_movies.append((row['movie_id'], row['title'], poster_url, match_count, row['clean_overviews']))

    # Sort the ranked list by match count (descending)
    ranked_movies.sort(key=lambda x: x[3], reverse=True)

    # Select the top 3 movies
    top_movies = ranked_movies[:3]

    # Format the recommendations for display
    recommendations = [{
        "id": movie[0],
        "title": movie[1],
        "poster_url": movie[2],
        "overview": movie[4],
        "emotion": emotion.capitalize(),
        "genres": ", ".join(genres)
    } for movie in top_movies]

    return recommendations

# Example usage
# Assuming the file 'movie_output.txt' is in the same directory as this script
with open('movie_output.txt', 'r') as file:
    recommendations = recommend_movies(file)

# Print the recommendations
for rec in recommendations:
    print(f"Movie ID: {rec['id']}, Title: {rec['title']}, Poster URL: {rec['poster_url']}, Emotion: {rec['emotion']}, Genres: {rec['genres']}, Overview: {rec['overview']}")


Movie ID: 2059, Title: National Treasure, Poster URL: https://image.tmdb.org/t/p/w300/pxL6E4GBOPUG6CdkO9cUQN5VMwI.jpg, Emotion: Neutral, Genres: adventure, mystery, action, Overview: modern treasure hunter led archaeologist ben gate search chest rich rumored stashed away george washington thomas jefferson benjamin franklin revolutionary war chest whereabouts may lie secret clue embedded constitution declaration independence gate race find gold enemy
Movie ID: 262504, Title: Allegiant, Poster URL: https://image.tmdb.org/t/p/w300/ijpRQOwZzGy0vb3Uqmk2Naug1C0.jpg, Emotion: Neutral, Genres: adventure, mystery, action, Overview: beatrice prior tobias eaton venture world outside fence taken protective custody mysterious agency known bureau genetic welfare
Movie ID: 58574, Title: Sherlock Holmes: A Game of Shadows, Poster URL: https://image.tmdb.org/t/p/w300/y1MYZkwhZK6L0Jy4YMuPktzDOfn.jpg, Emotion: Neutral, Genres: adventure, mystery, action, Overview: new criminal mastermind large professor 